# Get Galaxies From Halos

info: https://cdcvs.fnal.gov/redmine/projects/des-clusters/wiki/ClustDataVecY3 <br>
path: /data/des81.b/data/mariaeli/y3_buzz/Buzzard-3_v2.0_Y3a_mastercat/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from astropy.table import Table, vstack
from astropy.io.fits import getdata

import pandas as pd

import esutil

## Load

In [2]:
import h5py

#path   = '/data/des81.b/data/mariaeli/y3_buzz/Buzzard-3_v2.0_Y3a_mastercat/'
path   = '/global/project/projectdirs/des/jderose/Chinchilla/Herd/Chinchilla-3/v1.9.8/sampleselection/Y3a/mastercat/'
fname  = path+'Buzzard_v1.9.8_Y3a_gold.h5'
fname_aux  = path+'Buzzard-3_v1.9.8_Y3a_mastercat.h5'

master = h5py.File(fname,'r')
indexes= h5py.File(fname_aux,'r')

In [3]:
ls /global/project/projectdirs/des/jderose/Chinchilla/Herd/Chinchilla-3/v1.9.8/sampleselection/Y3a/mastercat/

Buzzard-3_v1.9.8_Y3a_mastercat.h5  Buzzard_v1.9.8_Y3a_shape.h5
Buzzard-3_v2.0_Y3a_mastercat.h5    Buzzard_v2.0_Y3a_gold.h5
Buzzard_v1.9.8_Y3a_bpz.h5          Y3_GOLD_2_2_1_maps_copy.h5
Buzzard_v1.9.8_Y3a_gold.h5         buzzard-1.9.8_3y3a_run_redmapper_v6.4.22.h5


In [4]:
def show_h5_group(name,node):
    ''' show all the h5 groups
    Example:
    fmaster = h5py.File(copa.master_fname,'r')
    fmaster.visititems(show_h5_group)

    ## https://stackoverflow.com/questions/45562169/traverse-hdf5-file-tree-and-continue-after-return
    '''
    if isinstance(node, h5py.Group):
        print(name,node)
    return None

def show_h5_dataset(name,node):
    if isinstance(node, h5py.Dataset):
        print(name,node)
    return None

In [5]:
columns = ['hpix_16384','haloid','coadd_object_id','ra','dec','rhalo']
columns+= ['mag_%s'%(ix) for ix in ['g','r','i','z']]
columns+= ['mag_err_%s'%(ix) for ix in ['g','r','i','z']]

In [6]:
columns

['hpix_16384',
 'haloid',
 'coadd_object_id',
 'ra',
 'dec',
 'rhalo',
 'mag_g',
 'mag_r',
 'mag_i',
 'mag_z',
 'mag_err_g',
 'mag_err_r',
 'mag_err_i',
 'mag_err_z']

In [7]:
master.visititems(show_h5_dataset)

catalog/gold/coadd_object_id <HDF5 dataset "coadd_object_id": shape (1505017869,), type ">i8">
catalog/gold/dec <HDF5 dataset "dec": shape (1505017869,), type ">f4">
catalog/gold/flags_gold <HDF5 dataset "flags_gold": shape (1505017869,), type ">i8">
catalog/gold/flux_g <HDF5 dataset "flux_g": shape (1505017869,), type ">f4">
catalog/gold/flux_i <HDF5 dataset "flux_i": shape (1505017869,), type ">f4">
catalog/gold/flux_r <HDF5 dataset "flux_r": shape (1505017869,), type ">f4">
catalog/gold/flux_z <HDF5 dataset "flux_z": shape (1505017869,), type ">f4">
catalog/gold/haloid <HDF5 dataset "haloid": shape (1505017869,), type ">i8">
catalog/gold/hpix_16384 <HDF5 dataset "hpix_16384": shape (1505017869,), type ">i8">
catalog/gold/ivar_g <HDF5 dataset "ivar_g": shape (1505017869,), type ">f4">
catalog/gold/ivar_i <HDF5 dataset "ivar_i": shape (1505017869,), type ">f4">
catalog/gold/ivar_r <HDF5 dataset "ivar_r": shape (1505017869,), type ">f4">
catalog/gold/ivar_z <HDF5 dataset "ivar_z": shap

In [8]:
indexes['index'].visititems(show_h5_dataset)

bpz/match_gold <HDF5 dataset "match_gold": shape (1505017869,), type "<i8">
bpz/select <HDF5 dataset "select": shape (1505017869,), type "<i8">
catalog/maglim/select <HDF5 dataset "select": shape (13438886,), type "<i8">
coadd_object_id <HDF5 dataset "coadd_object_id": shape (1505017869,), type "<i8">
gold/select <HDF5 dataset "select": shape (1505017869,), type "<i8">
maglim/random_select <HDF5 dataset "random_select": shape (250492120,), type "<i8">
maglim/select <HDF5 dataset "select": shape (12524606,), type "<i8">
mask/fracgood <HDF5 dataset "fracgood": shape (21007033,), type "<i8">
mask/hpix <HDF5 dataset "hpix": shape (21007033,), type "<i8">
mask/maskfrac_est <HDF5 dataset "maskfrac_est": shape (21007033,), type "<i8">
mask/zmax <HDF5 dataset "zmax": shape (21007033,), type "<i8">
metacal/match_gold <HDF5 dataset "match_gold": shape (1505017869,), type "<i8">
randoms/maglim/select <HDF5 dataset "select": shape (268777720,), type "<i8">
redmagic/combined_sample_fid/match_gold <

In [9]:
select     = indexes['index/gold/select'][:]
mag_i      = master['catalog/gold/mag_i'][:][select]
maglim_idx = select[np.where(mag_i<=24.)]
hpx16384   = master['catalog/gold/hpix_16384'][:][maglim_idx]
#mids       = master['catalog/gold/coadd_object_id'][:][maglim_idx]

#maglim_idx = select[np.where(mag_i<=24.)]
select=mag_i=0

master.close()
indexes.close()

### Loading Data

In [10]:
## Healpix map
Nside    = 2
outdir   = '/global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/'
hpx_file = outdir+'tiles/hpxmap_nside%i_buzzard_y3_v2.0.0.fits'%Nside

In [11]:
hpx_map = Table(getdata(hpx_file))
tiles   = np.where(hpx_map['hpx_value']>0)[0]

In [12]:
tiles

array([ 0, 16, 17, 18, 19, 20, 22, 32, 33, 34, 35, 38, 44, 45, 46, 47])

In [13]:
## Get Cluster Catalog

outdir = '/global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/'
name   = outdir+'halos/buzzard_y3_v2.0.0_golden_halos.fits'

cat = Table(getdata(name))

In [14]:
cat

haloid,coadd_object_id,RA,DEC,M200,R200,MAG_I,redshift,magLim,hpx8,hpx4,hpx2
int64,int64,float32,float32,float32,float32,float32,float64,float64,int64,int64,int64
12109345,34000000190,87.67945,-31.286345,53750000000000.0,0.811507,15.644118,0.12207666344755042,18.727379942055908,80,80,20
7921723,106000000031,17.458162,-37.26816,54520000000000.0,0.823985,15.9647045,0.15707450057296524,19.299043008021513,138,138,34
12127773,41000000098,84.258545,-38.53852,52040000000000.0,0.806834,16.234589,0.1376620470619489,19.00025480005313,133,133,33
10295050,5000000068,44.179226,-15.528681,50970000000000.0,0.802234,16.17392,0.14417593382404809,19.10681494118477,143,143,35
10559488,359000989610,29.339111,3.0369565,59480000000000.0,0.844769,16.401712,0.14416136199711516,19.106581791953843,69,69,17
8146031,369000000069,15.199777,-2.195923,54050000000000.0,0.822436,16.283852,0.15945146239187055,19.332320473486188,70,70,17
10030365,48006301558,59.113914,-41.401073,58910000000000.0,0.843198,16.556578,0.14710242891679756,19.15363886266876,134,134,33
10049271,53006584760,35.619694,-61.57988,53050000000000.0,0.814796,16.727932,0.15048246889904238,19.206754564586593,130,130,32
10294725,7000000067,51.846664,-16.289898,51970000000000.0,0.80679,17.323595,0.13937409684592666,19.02935964638075,141,141,35


In [ ]:
# ra_min,ra_max   = 26.,30.
# dec_min,dec_max = -44.5,-43.

# mask = (cat['RA']<ra_max)&(cat['RA']>ra_min)
# mask&= (cat['DEC']<dec_max)&(cat['DEC']>dec_min)

### Healpix Matching

In [15]:
import sys
# sys.path.append('/home/s1/jesteves/git/buzzardAnalysis/buzzardSelection/scripts/')
sys.path.append('/global/u2/j/jesteves/codes/buzzardAnalysis/buzzardSelection/scripts/')
from helper import AngularDistance,get_healpix_list

In [16]:
rad2deg  = 180/np.pi
rmax     = 8 #Mpc around each cluster

r200     = cat['R200'][:]*0.7 ## Mpc
DA       = AngularDistance(np.array(cat['redshift']))

## compute the radii max
cat['rmax']  = 60*(float(rmax)/DA)*rad2deg ## arcmin

In [17]:
Nside = 16384
cluster_tile = cat['hpx2']

### Retrieving Galaxy Catalogs

In [18]:
def save_hdf5_output(gal,cat,outfile):
    df  = gal.to_pandas()
    df.to_hdf(outfile, key='members', mode='w')

    gal = 0

    dfc = cat.to_pandas()
    dfc.to_hdf(outfile, key='cluster', mode='a')

In [19]:
def load_hdf_files(infile,indices,columns,path='catalog/gold/'):
    master = h5py.File(infile,'r')
    gold   = master[path]
    
    mydict = dict().fromkeys(columns)
    for col in columns:
        mydict[col] = gold[col][:][indices]
    
    master.close()
    return Table(mydict)

In [21]:
from time import time

In [22]:
fname  = path+'Buzzard_v1.9.8_Y3a_bpz.h5'
indexes= h5py.File(fname)
bpz    = indexes['catalog/bpz']

#b_cid  = bpz['coadd_object_id'][:][maglim_idx]
z      = bpz['z'][:][maglim_idx]
zmean  = bpz['zmean_sof'][:][maglim_idx]

indexes.close()

In [24]:
indexes= h5py.File(fname_aux,'r')
dnf    = indexes['catalog/dnf/unsheared']

#d_cid  = dnf['coadd_object_id'][:][maglim_idx]
#d_z    = dnf['z'][:][maglim_idx]
d_zmean= dnf['z_mean'][:][maglim_idx]
d_sigma= dnf['z_sigma'][:][maglim_idx]

indexes.close()

In [ ]:
## BE CAREFULL! The cells below take about an hour to run

In [25]:
outdir = '/global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/tiles/'
outfile_base = outdir+'buzzard_y3_v2.0.0_golden_{}.hdf'

In [26]:
fname  = path+'Buzzard_v1.9.8_Y3a_gold.h5'

t0 = time()

time_evolution = np.empty((len(tiles)+1,),dtype=np.float64)
time_evolution[0] = t0

for i, tile in enumerate(tiles):
    print('start tile: %i'%(tile))
    
    w      = esutil.numpy_util.where1(cluster_tile==tile)
    circles= get_healpix_list(cat[w],nside=Nside)
    
    match  = esutil.numpy_util.match(circles,hpx16384)
    indices= maglim_idx[match[1]]
    
    data        = load_hdf_files(fname,indices,columns,path='catalog/gold/')
    data['z']          = z[match[1]]
    data['z_mean_bpz'] = zmean[match[1]]

    data['z_mean_dnf'] = d_zmean[match[1]]
    data['z_sigma_dnf']= d_sigma[match[1]]
    
    data['index']= indices
    data['tile'] = tile
    time_evolution[i+1] = time()
    
    outfile = outfile_base.format(tile)
    save_hdf5_output(data,cat[w],outfile)
    data = 0
    print('outfile saved: %s'%(outfile))
    
    print('partial time: %.2f min'%((time_evolution[i+1]-time_evolution[i])/60.))
    print('run time    : %.2f min'%((time_evolution[i+1]-time_evolution[0])/60.))
    print('\n')

start tile: 0
outfile saved: /global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/tiles/buzzard_y3_v2.0.0_golden_0.hdf
partial time: 3.70 min
run time    : 3.70 min


start tile: 16
outfile saved: /global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/tiles/buzzard_y3_v2.0.0_golden_16.hdf
partial time: 3.66 min
run time    : 7.35 min


start tile: 17
outfile saved: /global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/tiles/buzzard_y3_v2.0.0_golden_17.hdf
partial time: 3.95 min
run time    : 11.30 min


start tile: 18
outfile saved: /global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/tiles/buzzard_y3_v2.0.0_golden_18.hdf
partial time: 3.53 min
run time    : 14.83 min


start tile: 19
outfile saved: /global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/tiles/buzzard_y3_v2.0.0_golden_19.hdf
partial time: 3.66 min
run time    : 18.49 min


start tile: 20
outfile saved: /global/project/projectdirs/des/jesteves/buzzardS

In [ ]:
## In case that the memory crashes, delete the photoz variables (z,zmean,d_zmean,d_sigma) and run the cell below

In [ ]:
# fname     = path+'Buzzard_v1.9.8_Y3a_gold.h5'
# fname_bpz = path+'Buzzard_v1.9.8_Y3a_bpz.h5'
# fname_dnf = fname_aux

# t0 = time()

# time_evolution = np.empty((len(tiles)+1,),dtype=np.float64)
# time_evolution[0] = t0

# for i, tile in enumerate(tiles):
#     print('start tile: %i'%(tile))
    
#     w      = esutil.numpy_util.where1(cluster_tile==tile)
#     circles= get_healpix_list(cat[w],nside=Nside)
    
#     match  = esutil.numpy_util.match(circles,hpx16384)
#     indices= maglim_idx[match[1]]
    
#     data = load_hdf_files(fname,indices,columns,path='catalog/gold/')
#     bpz  = load_hdf_files(fname_bpz,indices,['z','zmean_sof'],path='catalog/bpz/')
#     dnf  = load_hdf_files(fname_aux,indices,['z_mean','z_sigma'],path='catalog/dnf/unsheared')
    
#     data['z']          = bpz['z']
#     data['z_mean_bpz'] = bpz['zmean_sof']

#     data['z_mean_dnf'] = dnf['z_mean']
#     data['z_sigma_dnf']= dnf['z_sigma']
    
#     data['index']= indices
#     data['tile'] = tile
#     time_evolution[i+1] = time()
    
#     outfile = outfile_base.format(tile)
#     save_hdf5_output(data,cat[w],outfile)
#     data = 0
#     print('outfile saved: %s'%(outfile))
    
#     print('partial time: %.2f min'%((time_evolution[i+1]-time_evolution[i])/60.))
#     print('run time    : %.2f min'%((time_evolution[i+1]-time_evolution[0])/60.))
#     print('\n')

In [ ]:
# t0 = time()

# time_evolution = np.empty((len(tiles)+1,),dtype=np.float64)
# time_evolution[0] = t0

# for i, tile in enumerate(tiles):
#     print('start tile: %i'%(tile))
    
#     w      = esutil.numpy_util.where1(cluster_tile==tile)
#     circles= get_healpix_list(cat[w],nside=Nside)
    
#     match  = esutil.numpy_util.match(circles,hpx16384)
#     indices= maglim_idx[match[1]]
    
#     data        = load_hdf_files(fname,indices,columns,path='catalog/gold/')
#     data['row'] = indices
    
#     time_evolution[i+1] = time()
#     outfile = outfile_base.format(tile)
#     save_hdf5_output(data,cat[w],outfile)
    
#     data = 0
#     print('outfile saved: %s'%(outfile))
    
#     print('tile proc. time: %.2f min'%((time_evolution[i+1]-time_evolution[i])/60.))
#     print('partial time   : %.2f min'%((time_evolution[i+1]-time_evolution[0])/60.))
#     print('\n')

## Get the redshift

In [ ]:
outdir = '/global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/tiles/'
outfile_base = outdir+'backup/buzzard_y3_v2.0.0_golden_{}.hdf'

In [ ]:
def upload_dataFrame(infile,keys='members'):
    hdf = pd.HDFStore(infile, mode='r')
    df1 = hdf.get(keys)
    hdf.close()
    data = Table.from_pandas(df1)
    return data


In [ ]:
a = np.full(10,-1)

In [ ]:
a

In [ ]:
def init_columns(data,columns):
    nsize = len(data)
    for col in columns:
        data[col] = np.full(nsize,-99,dtype=np.float64)
    return data

In [ ]:
def update_hdf5_output(gal,outfile):
    df  = gal.to_pandas()
    df.to_hdf(outfile, key='members', mode='a')
    

In [ ]:
from time import time

In [ ]:
for tile in tiles:
    t0 = time()
    print('Tile: %i'%tile)
    data    = upload_dataFrame(outfile_base.format(tile))
    cdata   = upload_dataFrame(outfile_base.format(tile),keys='cluster')
    
    data    = init_columns(data,['z','z_mean_bpz','z_mean_dnf','z_sigma_dnf'])

    gcid    = np.array(data['coadd_object_id'])

    matchd= esutil.numpy_util.match(gcid,d_cid)
    matchb= esutil.numpy_util.match(gcid,b_cid)

    data['z'][matchb[0]]          = z[matchb[1]]
    data['z_mean_bpz'][matchb[0]] = zmean[matchb[1]]

    data['z_mean_dnf'][matchd[0]] = d_zmean[matchd[1]]
    data['z_sigma_dnf'][matchd[0]]= d_sigma[matchd[1]]
    
    outfile = outdir+'buzzard_y3_v2.0.0_golden_{}.hdf'.format(tile)
    print(outfile)
    print('\n')
    save_hdf5_output(data,cdata,outfile)
    print('partial time: %.2f s'%(time()-t0))
    print('\n')

In [ ]:
df  = gal.to_pandas()
df.to_hdf('bla', key='members', mode='a')


In [ ]:
match[0].size

In [ ]:
gcid.size

In [ ]:
b_cid = bcid

In [ ]:
indexes.close()

In [ ]:
fname  = path+'Buzzard_v1.9.8_Y3a_bpz.h5'
indexes= h5py.File(fname)
bpz    = indexes['catalog/bpz']

b_cid = bpz['coadd_object_id'][:][maglim_idx]
z    = bpz['z'][:][maglim_idx]
zmean  = bpz['zmean_sof'][:][maglim_idx]

indexes.close()

In [ ]:
dnf.keys()

In [ ]:
indexes= h5py.File(fname_aux,'r')
dnf    = indexes['catalog/dnf/unsheared']

d_cid  = dnf['coadd_object_id'][:][maglim_idx]
d_z    = dnf['z'][:][maglim_idx]
d_zmean= dnf['z_mean'][:][maglim_idx]
d_sigma= dnf['z_sigma'][:][maglim_idx]

indexes.close()

In [ ]:
#indexes.visititems(show_h5_dataset)

In [ ]:
dn = d_cid[np.isin(maglim_idx,indices)]

In [ ]:
dn-gcid

In [ ]:
data

In [ ]:
import esutil
match= esutil.numpy_util.match(cid,bcid)
cid  = cat['coadd_object_id'][match[0]]

In [ ]:
# bcid = bpz['catalog/bpz/coadd_object_id'][:][maglim_idx[select]]
# cid  = cat['coadd_object_id'][:]indexes

In [ ]:
plt.plot(bcid[match[1]],cid)

In [ ]:
# z = bpz['catalog/bpz/z'][:][maglim_idx[select]]

In [ ]:
cat['redshift'][match[0]] = z[match[1]]

In [ ]:
plt.hist(z,bins=np.linspace(0.09,1.,21))
_ = plt.xticks(np.arange(0.1,1.,0.2))

### Save Output

In [ ]:
outdir = '/global/project/projectdirs/des/jesteves/buzzardSelection/y3_v2.0.0/'
name   = outdir+'halos/buzzard_y3_v2.0.0_golden_halos.fits'

In [ ]:
cat_golden.write(name,format='fits')

In [ ]:
len(cat_golden)

In [ ]:
def sky_plot(RA,DEC,ax=None,title=''):
    if ax is None:
        fig = plt.figure(figsize=(8,6))
        ax = fig.add_subplot(111, projection="aitoff")

    ############################
    #Codigo para plotar coordenadas de objetos na esfera celeste
    #############################
    import astropy.coordinates as coord
    from astropy import units as u
    
    ra = coord.Angle(RA*u.degree)
    ra = ra.wrap_at(180*u.degree)
    dec = coord.Angle(DEC*u.degree)

    ##############
    #Plotando os objetos
    #import astropy.coordinates as coord
    ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
    ax.grid(True)
    ax.scatter(ra.radian, dec.radian, s=10, alpha=0.5)
    ax.set_title(title,fontsize=20)
    
    plt.subplots_adjust(top=0.9,bottom=0.0)
    # ax.set_xticklabels(['10h','8h','6h','4h','2h','0h','20h','18h','16h','14h','12h'])
    
    #fig.savefig(savefig, bbox_inches = "tight")
    #plt.clf()
    #plt.close()

In [ ]:
# ra_min,ra_max   = 26.,30.
# dec_min,dec_max = -44.5,-43.

# cut = (data['ra']<ra_max)&(data['ra']>ra_min)
# cut&= (data['dec']<dec_max)&(data['dec']>dec_min)

# data = data[cut]

In [ ]:
#w, = np.where((data['ra']<28)&(data['dec']<-42))
#w, = np.where(cut)
# plt.scatter(data['ra'][::],data['dec'][::],s=1)